**About this notebook.** This notebook was originally taken from https://github.com/fchollet/deep-learning-with-python-notebooks, a GitHub repository that holds the examples used in [Deep Learning with Python by François Chollet](https://www.manning.com/books/deep-learning-with-python) (An excellent and popular resource for getting started.

This notebook has been modified by [John Simpson](mailto:john.simpson@ualberta.ca) to support a workshop.  These modifications are mostly supplementations to help workshop participants get up to speed quickly given that they will not have direct access to the book and the workshop is relatively short.

**About Keras.** Keras is a popular Python Library that makes machine/deep learning easier by supplying what is known as an application programming interface (API) for the core pieces of code that actually do the work.  We'll start by loading Keras and checking the version that is installed.

Note that importing keras requires that the kera software be installed in the system already.  This is similar to the idea that to open a book you must have already gotten the book out of the library.  So, we will run the command to do this first.

In [ ]:
!pip install keras matplotlib

In [ ]:
import keras
keras.__version__

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

We will now take a look at a first concrete example of a neural network, which makes use of the Python library Keras to learn to classify 
hand-written digits. Unless you already have experience with Keras or similar libraries, you will not understand everything about this 
first example right away. You probably haven't even installed Keras yet. Don't worry, that is perfectly fine. In the next chapter, we will 
review each element in our example and explain them in detail. So don't worry if some steps seem arbitrary or look like magic to you! 
We've got to start somewhere.

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays. Numpy (sometimes written as NumPy) is an abbreviation for "Numerical Python" and is the name given to a library that gives Python more sophisticated mathematical abilities.  Of particular interest here is that it gives Python arrays, rectangular lists of data all of the same type (think of an Excel spreadsheet of any given size where every cell has some value in it that is the same kind of data as every other cell in the spreadsheet).  Putting data into standard sizes and knowing that every element in it is of the same type allows for some very fast mathematical operations to be carried out.

We load the MNIST dataset as follows:

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

The "load_data()" method divides the MNIST data into data to be used to train the model and data used to test the model.  Each of these categories is further divided. `train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the "test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.

Let's have a look at the training data.

## Investigating the Data

It is generally a good idea to check that the data being worked with is what you expect by looking at a sample of elements.  Let's look at the first element from `train_images`.

In [ ]:
train_images[0]

What we are seeing is the conent of a numpy array filled with integers.  The zeros correspond to areas where the original image was "empty" and the other numbers where the image was seen to not be empty.  If you were to count them you would see that there are 28 rows (everything between the interior square bracket is a row) each holding 28 values.  We can see at the end of the information that `dtype=uint8` which means that every value in the array is an 8-bit unsigned integer (this doesn't really matter for us, but it can in other programming situations).

While this is what the underlying data really looks like it is ultimately not a very satisfying view.  Counting rows and the values they contain is tedious and with this output format doesn't really allow to confirm what we are looking at.

We can check the dimensions by using numpy's ability to give us the "shape" of its arrays.

In [ ]:
train_images[0].shape

So, we can see that the array really does have 28 rows and 28 columns, making it a 28x28 array of integers.

To have python show the content of the array as an image we need to load a library that is meant for plotting graphs and charts called `matplotlib`, the math plotting library.  We can use this library to produe what amounts to a heatmap of the 28x28 array.  Locations within the array with low values will get a dark colour and locations with higher values will get lighter colours.

We'll even go a step further and print out the label that corresponds to each image so that we can verify that the matching of images to labels seems sensible in our training data.

Note that the code below goes beyond what is necessary by actually asking for input rather than requiring tweaking.  It is expected that this cell will be run more than once and the intent is to make this very easy to do.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 
# The line of code above is not Python but an instruction to the notebook.
# It tells the notebook to render any images produced with matplotlib as part of the notebook
# rather than in a separate window.

index = int(input("What index from the training set would you like to see (0-59999): "))
plt.imshow(train_images[index])
print("\nThe training label for index =",index,"is '",train_labels[index],
      "'. The image for the same index is:")

So the individual images in the training array look good.  Let's check the shape of the overall array that each 28x28 array is held in:

In [ ]:
train_images.shape

We can see it is a 60000x28x28 array.  Or, to put it another way, there are 60,000 sample images where each image is stored as an arrray of 28 pixel values by 28 pixel values.

Let's try to look at train_labels directly:

In [ ]:
train_labels

What we are seeing here is a truncated version of the entire array.  Why? 60000 x 28 x 28 = 47,040,000.  This is much too large to usefully display in the notebook and so it is automatically truncated.  This is no big deal, we can use the techniques already shown to investigate inside.

In an identical fashion we can look at the test data:

In [ ]:
test_images.shape

In [ ]:
test_labels

In [ ]:
len(test_labels)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 

index = int(input("What index from the training set would you like to see (0-9999): "))
plt.imshow(test_images[index])
print("\nThe testing label for index =",index,"is '",test_labels[index],
      "'. The image for the same index is:")

## Training the neural network

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

[Note that a warning message may be displayed.  This is there because keras is making use of a feature of tensorflow that will not exist in the future.  If you have an older version of tensorflow installed then you may not see this message because that version doesn't "know" about this deprecation.  This warning doesn't matter for us right now but if you come back to this notebook in the future this may become important.]


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two layers (Three if we count the input layer but the input layer is usually not counted).

The first layer is 512x512 and because we cannot see it during normal processing it is called a "hidden layer".  In fact, even if we could see it during normal processing it would appear as nonsense to us.  Many networks often have many layers with many shapes and it is this stacking of layers that makes the learning "deep".  This is a rectified linear unit (ReLU or relu) layer, meaning that however all the inputs combine to for a new value the output is the maximum of either 0 or that new value.

The second layer is 1x10 and because it is the last layer it is the output layer.  This is a "softmax" layer, the values that this layer can take all fall between 0 and 1 but the sum of all the cells in this layer must be 1.  Each score will be the probability that the current image belongs to one of our 10 digit classes.  

All the layers in this model are densely-connected (also called "fully-connected") to the layers beside them. This means that each cell in an adjacent layer receives input from each cell in the layer above and/or provides input to each cell in the layer below.

Each of the 724 cells in the input layer thus provides input to eart of the 512 x 512 = 262,144 cells in the hidden layer.  Each cell in the hidden provides input to each of the 10 cells in the output layer.  Across the entire model there are 208,142,336 connections, each which amounts to a weighting on the value of some input from the layer above to the layer below.  Even in this very simple model we can see why it is often said that exactly how a neural network "sees" the world is not available to us and is a black box.


![](https://github.com/ualberta-rcg/ai-ethics-workshop/raw/master/notebooks/layersDiagram.jpeg)

Visit [THIS PAGE](http://scs.ryerson.ca/~aharley/vis/conv/flat.html) by [Adam Harley](https://www.cs.cmu.edu/~aharley/) for a visualization about how layers connect.  Note that the network used in this visualization is much more complicated than the one we are using/looking at here, but you will quickly get the idea.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: this is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly classified).

We're going to ignore the details of both the loss function and the optimizer here.

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in 
the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with 
values in the `[0, 255]` interval. We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1.

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to categorically encode the labels, which amounts to assigning each a specific value:

In [ ]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network:  we "fit" the model to its training data.

DO NOT RUN THE CELLS BELOW!

As with training the Shakespeare model in Part 1 it will simply take too long.  The output from similar runs has been pasted in below.

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

Each epoch is a run through the entire set of input data.  At each step the network is presented with a set of images from the training set which it then attempts to classify.  The results of this are then compared with the preassigned label.  The loss function track these results and feeds them back to the optimizer which then modifies the network slightly in an attempt to improve the results on the next trial.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

In [ ]:
print('test_acc:', test_acc)


Our test set accuracy turns out to be 97.8% -- that's quite a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 
Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in 
less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.